In [31]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import sympy as sp
import math
import re
from random import random, randint


class Func:
    """
        variables is [xi for i in range(number_of_variables)]
    """

    def __init__(self, number_of_variables: int, function_string, *args):
        sp.init_printing(use_unicode=True)
        self.sp_variables = sp.symbols("x:" + str(number_of_variables))
        self.string_variables = ["x" + str(i) for i in range(number_of_variables)]
        self.f = sp.sympify(function_string)

    def diff(self, variable):
        return self.f.diff(self.sp_variables[self.string_variables.index(variable)])

    """
        [("x0", 1), ("x1", 2)] ->  [(x0, 1), (x1, 2)]
        where x1 x2 is sp.symbols
    """

    def _parse_arguments(self, l):
        res = [(self.sp_variables[self.string_variables.index(variable)], value) for variable, value in l]
        return res

    # unused
    # def _parse_function_string(self, s:str):
    #     import re
    #     delimiters = "^", "+", "(c)", " ", "\n", '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', "**", "sin", "cos", "(", ")", "e"
    #     regex_pattern = '|'.join(map(re.escape, delimiters))
    #     return [i for i in re.split(regex_pattern, s) if i != ""]

    """
        variable_value = [("x0", 1), ("x1", 2)]
        variable_value = value
    """

    def eval(self, variable_value):
        return self.f.subs(self._parse_arguments(variable_value))

    """
        without vectors like grad("0.5x^2 + bx + c") = ax + b
        without vectors like grad("x^2 + y^2") = 2x + 2y
    """

    def grad(self):
        result = sp.sympify("0")
        for xi in self.sp_variables:
            result += self.f.diff(xi)
        return Func(len(self.sp_variables), str(result))

    """
        variable_value = [("x0", 1), ("x1", 2)]
        variable_value = value
    """

    def metric(self, variable_value):
        v = list(sp.ordered(self.f.free_symbols))
        gradient = lambda ff, v: sp.Matrix([ff]).jacobian(v)
        g = gradient(self.f, v).subs(self._parse_arguments(variable_value))
        res = 0
        for i in g:
            res += i * i
        return sp.sqrt(res).evalf()

    def __str__(self):
        return str(self.f)

In [33]:
from sympy import *

x, y, z = symbols('x y z')
init_printing(use_unicode=True)
"""
    examples
"""
f = Func(1, "x0^2 + 1")
print(f.eval([("x0", 1)]))
print(f.diff("x0"))
print(f.grad())

ff = Func(3, "x0^2 + x1**3 + sin(x2) + 1")
print(ff.eval([("x0", 1)]))
print(ff.diff("x0"))
print(ff.grad())


self.sp_variables = (x0,)
self.string_variables = ['x0']
2
2*x0
self.sp_variables = (x0,)
self.string_variables = ['x0']
2*x0
self.sp_variables = (x0, x1, x2)
self.string_variables = ['x0', 'x1', 'x2']
x1**3 + sin(x2) + 2
2*x0
self.sp_variables = (x0, x1, x2)
self.string_variables = ['x0', 'x1', 'x2']
2*x0 + 3*x1**2 + cos(x2)


In [164]:
import sympy as sp
from random import randint, random
from decimal import Decimal


class Quadratic_func():
    def __init__(self, n, A, b, c):
        sp.init_printing(use_unicode=True)
        if A == None:
            self.A = sp.eye(n)
            b = sp.Matrix([0 for i in range(n)])
            c = 0
        else:
            self.A = A
        self.list_variables = sp.symbols("x:" + str(n))
        self.v = sp.Matrix(list(sp.ordered(self.list_variables)))
        self._createFunc(n, self.A, b, c)

    def _createFunc(self, n, A, b, c):
        self.f = sp.sympify(0)
        for i in range(n):
            for j in range(n):
                self.f += A[i, j] * self.v[i] * self.v[j]
        for i in range(n):
            self.f += b[i] * self.v[i]
        self.f += c

    def __str__(self):
        return str(self.f)

    def cond(self):
        lambdas = self.get_lamdas()
        print(f"{lambdas = }")
        L = max([i for i in lambdas])
        l = min([i for i in lambdas])
        print(f"{L = } {l = }")
        return L / l

    """
        :return list of tuples [(lambda, gamma(lambda)]
        gamma(lambda) its power in det(A - tE)
    """

    def get_lamdas(self):
        return list(dict(self.A.eigenvals()).keys())


#fall if mi == ma
# 3 3.(3)
def create_random_quadratic_func(n: int, k: float):
    if k < 1:
        raise AssertionError("k must be >= 1")

    def get_vector(mi, ma, size):
        print(f"{mi = } {ma = }")
        if int(ma) - int(mi) == 1:
            res = [random() + mi for _ in range(size)]
        else:
            res = [random() + randint(int(mi), int(ma) - 1) for _ in range(size)]
        res[0] = mi
        res[-1] = ma
        return res

    # b and c must be anyone
    b = sp.Matrix([0 for _ in range(n)])
    c = 5
    if int(k) == 1:
        a_min = 1
    else:
        a_min = Decimal(randint(1, int(k) - 1))
    a_max = Decimal(k / a_min)
    a_max, a_min = max(a_max, a_min), min(a_max, a_min)
    return Quadratic_func(n, sp.diag(*get_vector(a_min, a_max, n)), b, c)





In [165]:
import sympy as sp

"""
    examples
"""
A = sp.Matrix([[1, 0],
               [0, 1]])
b = sp.Matrix([1, 1])
c = 5
qf = Quadratic_func(2, A, b, c)
k = 10

for i in range(10, 100):
    print(qf :=create_random_quadratic_func(i, k))
    print(qf.cond())


mi = Decimal('1.111111111111111111111111111') ma = Decimal('9')
1.11111111111111*x0**2 + 6.4907281100972*x1**2 + 3.37996123734115*x2**2 + 7.6357264340128*x3**2 + 7.21728538058842*x4**2 + 7.25710503777846*x5**2 + 8.60820276348052*x6**2 + 7.2982760294533*x7**2 + 5.7587333748926*x8**2 + 9.0*x9**2 + 5
lambdas = [1.11111111111111, 6.49072811009720, 3.37996123734115, 7.63572643401280, 7.21728538058842, 7.25710503777846, 8.60820276348052, 7.29827602945330, 5.75873337489260, 9.00000000000000]
L = 9.00000000000000 l = 1.11111111111111
8.10000000000000
mi = Decimal('2') ma = Decimal('5')
2.0*x0**2 + 4.5170122441123*x1**2 + 2.4047120198815*x10**2 + 3.01795738811682*x2**2 + 2.10799050806761*x3**2 + 4.22738575031479*x4**2 + 3.30582627686203*x5**2 + 2.12733079245503*x6**2 + 3.30801464972912*x7**2 + 2.66696938663286*x8**2 + 5.0*x9**2 + 5
lambdas = [2.00000000000000, 4.51701224411230, 2.40471201988150, 3.01795738811682, 2.10799050806761, 4.22738575031479, 3.30582627686203, 2.12733079245503, 3.30801464

ValueError: empty range for randrange() (3, 3, 0)